Despues de utilizar el metodo de dendogramas para explorar en cuantos clusters se puede separar los datos, usaremos ahora el metodo de K-Means.

In [21]:
import numpy as np
import pandas as pd
import math
import random
import time
import sys
from scipy.spatial import distance
np.set_printoptions(precision=2, suppress=True) # Cortar la impresión de decimales a 1

LARGER_DISTANCE = sys.maxsize
TALK = True # TALK = True, imprime resultados parciales



In [22]:
#Leemos el archivo
df=pd.read_csv('numeric_vgsales.csv')
del df['Unnamed: 0'] #borramos una columna que fue añadida erroneamente en los datos
del df['Exito']

In [23]:
DATA_SET = df.values
DATA_LEN = len(DATA_SET)

class DataPoint:
    def __init__(self, p):
        self.value = p[:]
        
    def set_value(self, p):
        self.value = p
    
    def get_value(self):
        return self.value
    
    def set_cluster(self, cluster):
        self.cluster = cluster
    
    def get_cluster(self):
        return self.cluster

data = []
def initialize_dataset():
    for i in range(DATA_LEN):
        point = DataPoint(DATA_SET[i])
        point.set_cluster(None)
        data.append(point)
    return
initialize_dataset()
NUM_CLUSTERS = 5

In [24]:
# Definir forma de muestreo; 0 = random, 1=head, 2=tail
SAMPLING_METHOD = 1 

centroids = []
def initialize_centroids():
    if (TALK) : 
        print("Centroides inicializados en:")
    for c in range(NUM_CLUSTERS):
        if (SAMPLING_METHOD == 0) :
            which = random.randint(0,DATA_LEN-1)
        elif (SAMPLING_METHOD == 1):
            which = c
        else :
            which = DATA_LEN-1 - c
                
        centroids.append(list(DATA_SET[which]))
        if (TALK) : 
            print(centroids[c])        
    if (TALK) : 
        print()
    
    return

# --------------------------
# Inicializar los centroides
initialize_centroids()

Centroides inicializados en:
[0.017101840543094545, 0.0078568018884454544, 0.0091680360032164645, 0.0026941311321977261, 0.01823301610861721, 9.0654365749241037e-05]
[0.0060136355869593282, 0.016586581764495959, 0.0091680360032164645, 0.0094294589626920406, 0.0059977026673082925, 9.0654365749241037e-05]
[0.017449475633331685, 0.0078568018884454544, 0.0091634588509432648, 0.0026941311321977261, 0.02015228096215586, 9.0654365749241037e-05]
[0.01744721826261586, 0.0034919119504202021, 0.0091909217645824558, 0.010776524528790904, 0.016553659361770886, 9.0654365749241037e-05]
[0.015393010911214587, 0.013094669814075756, 0.0091726131554896624, 0.01481772122708749, 0.000719724320076995, 9.0654365749241037e-05]



In [25]:


def update_clusters():
    changed = False
    
    for i in range(DATA_LEN):
        minDistance = LARGER_DISTANCE
        currentCluster = 0
        
        for j in range(NUM_CLUSTERS):
            dist = distance.euclidean(data[i].get_value(), centroids[j])
            if(dist < minDistance):
                minDistance = dist
                currentCluster = j
        
        if(data[i].get_cluster() is None or data[i].get_cluster() != currentCluster):
            data[i].set_cluster(currentCluster)
            changed = True
            
    members = [0] * NUM_CLUSTERS
    for i in range(DATA_LEN):
        members[data[i].get_cluster()] += 1
    
    if (TALK) : 
        for j in range(NUM_CLUSTERS):
            print("El cluster ", j, " incluye ", members[j], "miembros.")
        print()
            
    return changed

# --------------------------
# Actualizar los clusters
KEEP_WALKING = update_clusters()



El cluster  0  incluye  429 miembros.
El cluster  1  incluye  7014 miembros.
El cluster  2  incluye  602 miembros.
El cluster  3  incluye  852 miembros.
El cluster  4  incluye  2964 miembros.



In [26]:
def update_centroids():    
    if (TALK) : 
        print("Los nuevos centroids son:")
    for j in range(NUM_CLUSTERS):
        means = [0] * DATA_SET.shape[1]
            
        clusterSize = 0
        for k in range(len(data)):
            if(data[k].get_cluster() == j):
                p = data[k].get_value()
                for i in range(DATA_SET.shape[1]):
                    means[i] += p[i]
                clusterSize += 1

        if(clusterSize > 0):
            for i in range(DATA_SET.shape[1]):
                centroids[j][i] = means[i] / clusterSize

        if (TALK) : 
            print(centroids[j])        
    if (TALK) : 
        print()
    
    return

# --------------------------
# Actualizar los centroides
update_centroids()

Los nuevos centroids son:
[0.012368328843655164, 0.0073053402692357531, 0.0091788547267712042, 0.0034634343109721227, 0.012965103835792576, 0.00079095405827368098]
[0.0046797161598623729, 0.0088155343433237391, 0.009180193452154194, 0.0070113091847120949, 0.002301342627391944, 0.0050188836663696288]
[0.013632445492347505, 0.0080337176932425113, 0.0091833337280895506, 0.0070374106401676411, 0.030701662102199167, 0.0005251026135674481]
[0.012537749596240379, 0.0041753348585570459, 0.009190325445448122, 0.010452406640234262, 0.014138950478038405, 0.00082216700016946639]
[0.011710720626787595, 0.00660447314239375, 0.0091840977308796767, 0.010486569120163729, 0.0019499818327861889, 0.00090204764137731389]



In [27]:
while(KEEP_WALKING):
    KEEP_WALKING = update_clusters()
    if (KEEP_WALKING):
        update_centroids()
    else :
        if (TALK) : 
            print ("No más cambios.")

El cluster  0  incluye  779 miembros.
El cluster  1  incluye  5800 miembros.
El cluster  2  incluye  641 miembros.
El cluster  3  incluye  693 miembros.
El cluster  4  incluye  3948 miembros.

Los nuevos centroids son:
[0.011683698771413975, 0.0083778991467719274, 0.0091790705243576087, 0.0039322555806274688, 0.011821094694837561, 0.00086441672501330234]
[0.0037160848399976089, 0.0086530180185191048, 0.0091804353508916844, 0.0069095173433518968, 0.0017451246588533779, 0.0058490978284563323]
[0.013710949258758947, 0.0078717827899488346, 0.0091849593275337739, 0.0083030515626468203, 0.031068286952465572, 0.00049909400425752256]
[0.011697929581430547, 0.0056586105632350798, 0.0091840461289165176, 0.01122943401926864, 0.014454030693532792, 0.0009964131369581083]
[0.011190156005490804, 0.0069033365686496035, 0.0091840606736116256, 0.0097884034853707325, 0.0022164154720101656, 0.00091779508586554092]

El cluster  0  incluye  907 miembros.
El cluster  1  incluye  5094 miembros.
El cluster  2 

El cluster  0  incluye  3867 miembros.
El cluster  1  incluye  242 miembros.
El cluster  2  incluye  677 miembros.
El cluster  3  incluye  1500 miembros.
El cluster  4  incluye  5575 miembros.

Los nuevos centroids son:
[0.0063336445926020684, 0.0085180257120024052, 0.0091775560538327767, 0.0030703480474774903, 0.0018529132320989339, 0.0030654163086037637]
[0.00029346752103548906, 0.0067024508304553291, 0.0091751286978546832, 0.007436692546727598, 0.00060125730044999757, 0.04262216149001196]
[0.013672987788207549, 0.0076118006806981161, 0.0091860336093481654, 0.0084166725917255436, 0.030636123476837182, 0.00050335267481742588]
[0.012059002281614115, 0.0073062437709291959, 0.0091806109662285595, 0.0080608403475355538, 0.01258709869508882, 0.00089101154282737348]
[0.007142721805322068, 0.0075805806920081599, 0.0091852813993596686, 0.011376723787299894, 0.0019874719116745299, 0.00275986037173528]

El cluster  0  incluye  3863 miembros.
El cluster  1  incluye  209 miembros.
El cluster  2  

El cluster  4  incluye  3672 miembros.

Los nuevos centroids son:
[0.0043271797868988295, 0.0084548168870514761, 0.0091778833000638223, 0.0028461824667962445, 0.0018311336903164314, 0.0043794728878413314]
[7.3703153871704978e-05, 0.004379439571152003, 0.0091719265826486866, 0.0070047409437140884, 0.00030188436758785104, 0.085997753161921722]
[0.012802942417168166, 0.0074294751321282793, 0.0091839465073399948, 0.0082497869346138553, 0.025242724089126419, 0.00067842347768512588]
[0.012341655006295726, 0.0074274432752904221, 0.009182055041857708, 0.0085648182349170616, 0.0038119100968475052, 0.00071707853922885309]
[0.0039664259618266357, 0.0079663996151027351, 0.0091848164018825396, 0.011476103514234871, 0.0020000182358566603, 0.0047527378156639148]

El cluster  0  incluye  2969 miembros.
El cluster  1  incluye  58 miembros.
El cluster  2  incluye  1192 miembros.
El cluster  3  incluye  3985 miembros.
El cluster  4  incluye  3657 miembros.

Los nuevos centroids son:
[0.004351161507512248

El cluster  1  incluye  55 miembros.
El cluster  2  incluye  1256 miembros.
El cluster  3  incluye  4130 miembros.
El cluster  4  incluye  3231 miembros.

Los nuevos centroids son:
[0.0054609931527699095, 0.0087284111429246913, 0.0091782997899539243, 0.0029940422491404281, 0.0020537006599542127, 0.0026757962180133433]
[7.1784388763253227e-05, 0.0042855283027884291, 0.0091694507593736378, 0.0070537251461176842, 0.00028788972803079829, 0.089652222942413068]
[0.012724956884840422, 0.0075961595752671914, 0.0091845079206328544, 0.0081360186181735263, 0.024677394816928676, 0.00068719762444149977]
[0.011785788258369623, 0.0071632811191154047, 0.0091830319953250892, 0.0096991982419083571, 0.0035473579432044179, 0.00080526539028490227]
[0.0029873433424326464, 0.0079664982867640299, 0.009183600587563661, 0.010731914186614356, 0.0017956353278058379, 0.0066787941548182763]

El cluster  0  incluye  3223 miembros.
El cluster  1  incluye  55 miembros.
El cluster  2  incluye  1259 miembros.
El cluster

El cluster  1  incluye  50 miembros.
El cluster  2  incluye  1257 miembros.
El cluster  3  incluye  5019 miembros.
El cluster  4  incluye  1677 miembros.

Los nuevos centroids son:
[0.0070292090015593662, 0.0089085389766745109, 0.009178757549378443, 0.0033875661281211225, 0.0024444136251863297, 0.0018948595267026364]
[6.7089057674336045e-05, 0.0042077539002563433, 0.0091693176058529613, 0.0071663888116459519, 0.00029268789016464487, 0.093607884985351303]
[0.012780453598501069, 0.0075588642936303607, 0.0091847242202953809, 0.0081477641201667689, 0.024663106855250441, 0.00067467907047267259]
[0.009163518241219094, 0.0073221267870508255, 0.0091852575500438911, 0.011331024919219007, 0.0030405807174243541, 0.0013474067296596699]
[0.0015207167516379134, 0.0072180755969777017, 0.0091781019171007661, 0.0085555130259505063, 0.0013743572933769923, 0.011705765248919441]

El cluster  0  incluye  3881 miembros.
El cluster  1  incluye  48 miembros.
El cluster  2  incluye  1260 miembros.
El cluster  

Antes de intentar con 5 clusters ya se habia intentado con 3 y 4 clusters, pero quedaban muy desbalanceados los datos y se requerian demasiadas iteraciones.